**CHALLENGE SPRINT**

A IA projetada para o sistema de recomendações personalizadas foi criada para fornecer informações para os usuários com base nos seus interesses pessoais, profissionais e acadêmicos. O sistema foi desenvolvido através de uma série de etapas que garantiram um bom funcionamento e eficácia, passando por etapas desde a coleta e preparação dos dados fornecidos até o treinamento e validação do modelo final. Dados como interesse profissional, interesse pessoal e área de interesse são coletados e armazenados em uma base de dados (Excel).





O primeiro passo é importar o módulo drive da biblioteca google.colab, permitindo o acesso ao Google Drive através do Google Colab

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**IMPORTS**

Para o melhor funcionamento do sistema foram realizados alguns imports (Pandas e Numpy) que realizam a manipulação e análise de dados e fornece suporte para arrays e operações matemáticas. Além disso, foram usados o TfidfVectorizer que transforma o texto bruto em um formato que os algoritmos de machine learning possam processar, a função Cosine_similarity que calcula a similaridade entre dois vetores, a função Train_test_split que divide os dados em conjuntos de treino e teste. o algoritimo Regressão Linear que modela a relação entre uma variável dependente e uma ou mais variáveis independentes e a função Mean_squared_error que calcula o erro entre os valores preditos pelo modelo e os valores reais.


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Em seguida, o código define o caminho do arquivo Excel onde os dados estão armazenados, os dados são carregados do arquivo Excel em um DataFrame e os espaços em branco dos nomes das colunas do DataFrame são removidos.

In [4]:
file_path = '/content/drive/MyDrive/Base de dados.xlsx'
data = pd.read_excel(file_path)

data.columns = data.columns.str.strip()

**PRÉ - PROCESSAMENTO**

A função preprocess_data(df) é definida, recebe o DataFrame e faz o pré-processamento dos dados. Logo em seguida, uma lista é definida com as colunas necessárias ('INTERESSE PROFISSIONAL', 'INTERESSE PESSOAL', 'AREA INTERESSE') e um loop é iniciado para verificar se todas as colunas necessárias estão presentes no DataFrame. Caso alguma coluna não seja encontrada, uma mensagem de erro é retornada.

Os valores ausentes nas colunas de interesse são preenchidas com strings vazias e uma nova coluna é criada combinando os interesses profissionais, pessoais e áreas de interesse. O código garante que a nova coluna "INTERESSE" não tenha valores nulos e em seguida retorna o DataFrame processado e aplica a função preprocess_data no DataFrame. Uma instância é criada do TfidfVectorizer para converter o texto de interesses em vetores numéricos e Transforma os textos da coluna "INTERESSE" em uma matriz de TF-IDF.




In [5]:
def preprocess_data(df):
    columns_needed = ['INTERESSE PROFISSIONAL', 'INTERESSE PESSOAL', 'AREA INTERESSE']
    for column in columns_needed:
        if column not in df.columns:
            print(f"Coluna não encontrada: {column}")
            return df

    df[columns_needed] = df[columns_needed].fillna('')

    df['INTERESSE'] = df['INTERESSE PROFISSIONAL'] + ' ' + df['INTERESSE PESSOAL'] + ' ' + df['AREA INTERESSE']

    df['INTERESSE'] = df['INTERESSE'].fillna('')

    return df

data = preprocess_data(data)

vectorizer = TfidfVectorizer()
interest_matrix = vectorizer.fit_transform(data['INTERESSE'])


**RECOMENDAÇÃO DE CURSOS**

Primeiramente a funcação para recomendar cursos é definida com base nos interesses do usuários e os interesses inseridos pelo usuário são combinados em uma string. Esses interesses são convertidos em um vetor numérico usando TF-IDF e a similaridade entre o vetor de interesses do usuário e todos os cursos disponíveis é calculada usando a similaridade cosseno.

Para armazenar as recomendações, uma lista vazia é criada e logo em seguida um loop percorre todas as similaridades calculadas entre os interesses do usuário e os cursos. Se a  similaridade for maior que 0, adiciona o curso à lista de recomendações e são armazenados o título do curso (TITULO), a nota do curso (NOTA) e a similaridade calculada. Em seguida, é verificado se a lista de recomendações contém algum curso após o término do loop e se e houver cursos recomendados, prossegue com a ordenação.  A lista de recomendações é ordenada com base em dois critérios: a similaridade (no índice x[2]) e a nota do curso (no índice x[1]). A função sorted() organiza os cursos pela maior similaridade e nota portanto recomendação de melhor curso é a primeira posição da lista ordenada. Depois de ordenar as recomendações, o código retorna o título (best_recommendation[0]) do curso com a maior similaridade e nota, recomendando-o ao usuário.

Caso nenhuma recomendação tenha sido encontrada, o código executa o bloco seguinte e retorna uma mensagem informando que não foram encontrados cursos que se alinhem com os interesses fornecidos.

In [6]:
def recommend_course(interesse_profissional, interesse_pessoal, area_interesse):
    user_interest = f"{interesse_profissional} {interesse_pessoal} {area_interesse}"
    user_interest_vector = vectorizer.transform([user_interest])

    course_similarities = cosine_similarity(user_interest_vector, interest_matrix).flatten()

    course_recommendations = []
    for i, similarity in enumerate(course_similarities):
        if similarity > 0:
            course_recommendations.append((data.iloc[i]['TITULO'], data.iloc[i]['NOTA'], similarity))

    if course_recommendations:
        best_recommendation = sorted(course_recommendations, key=lambda x: (x[2], x[1]), reverse=True)[0]
        return best_recommendation[0]
    else:
        return "Nenhum curso encontrado com base nos interesses fornecidos."

**ADICIONAR INFORMAÇÕES DO USUÁRIO**

Primeiramente a funçao é definida para adicionar informações do usuário ao DataFrame, depois essas informações são coletada por meio de inputs e um novo DataFrame com as informações inseridas. A nova entrada é adicionada e salva, atualizando o excel. Em seguida o código exibe uma mensagem de sucesso e encerra o programa após o registro das informações.

In [7]:
def add_user_info():
    user_info = {
        'ID USUÁRIO': input("Digite o seu ID: "),
        'NOME': input("Digite o seu nome: "),
        'E-MAIL': input("Digite seu e-mail: "),
        'SENHA': input("Digite sua senha: "),
        'DATA NASCIMENTO': input("Digite sua data de nascimento (YYYY-MM-DD): "),
        'CPF': input("Digite seu CPF: "),
        'PROFISSÃO': input("Digite sua profissão: "),
        'CATEGORIA DO CURSO': input("Digite a categoria do curso realizado: "),
        'TITULO': input("Digite o título do curso: "),
        'INTERESSE PROFISSIONAL': input("Digite seu interesse profissional: "),
        'INTERESSE PESSOAL': input("Digite seu interesse pessoal: "),
        'AREA INTERESSE': input("Digite sua área de interesse: "),
        'NOTA': float(input("Digite a nota do curso: ")),
        'COMENTARIO': input("Digite um comentário sobre o curso: ")
    }

    global data
    new_entry = pd.DataFrame([user_info])
    data = pd.concat([data, new_entry], ignore_index=True)
    data.to_excel(file_path, index=False)
    print("Informações do usuário registradas com sucesso!")

    print("Encerrando o programa...")
    exit()

**TREINAR O MODELO**

Para treinar o modelo de regressão linear, primeiro a função é definida e o pré-processamento aplicado. Os interesses são tranformados em vetores numéricos e as notas do curso são definidas como alvo.

Em seguida, os dados são divididos em conjuntos de treino e teste e um modelo de regressão linear é criado. Esse modelo é treinado com os dados de treino e faz previsões com os dados de teste, após essa etapa o erro quadrático médio das previsões é calculado e retornado.


In [8]:
def train_validate_model(df):
    df = preprocess_data(df)
    X = vectorizer.transform(df['INTERESSE'])
    y = df['NOTA']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse

**MAIN**

A função main é definida e criada para interação do usuário, onde um loop infinito é iniciado e solicita a opção do usuário. São fornecidas três opções:

*   Opção 1 = recomendação de cursos;
*   Opção 2 = adicionar informações do usuário;
*   Opção 3 = sair;

Se o usuário escolher a opção 1, ela chama função de recomendação de cursos, exibe o curso recomendado e treina e valida o modelo, calculando o MSE e depois retorna para o usuário.

Se o usuário escolher a opção 2, chama a função para adicionar informações do usuário e adiciona as informações na base de dados excel.

Se o usuário escolher a opção 3, encerra o programa e se a opção for inválida, exibe uma mensagem de erro.

In [ ]:
def main():
    while True:
        print("\nEscolha uma opção:")
        print("1. Obter recomendações de cursos")
        print("2. Registrar informações do usuário")
        print("3. Sair")

        choice = input("Digite o número da opção desejada: ")

        if choice == '1':
            interesse_profissional = input("Digite seu interesse profissional: ")
            interesse_pessoal = input("Digite seu interesse pessoal: ")
            area_interesse = input("Digite sua área de interesse: ")

            recommended_course = recommend_course(interesse_profissional, interesse_pessoal, area_interesse)
            print(f"Curso recomendado: {recommended_course}")

            mse = train_validate_model(data)
            print(f"Erro Quadrático Médio do Modelo: {mse}")

        elif choice == '2':
            add_user_info()

        elif choice == '3':
            print("Saindo...")
            break

        else:
            print("Opção inválida. Tente novamente.")

if __name__ == "__main__":
    main()



Escolha uma opção:
1. Obter recomendações de cursos
2. Registrar informações do usuário
3. Sair
